# Digital Musicology (DH-401)
## Assignment 3: Similarity
Group 6
- Mickaël Achkar
- Yichen Wang
- Yinghui Jiang

In [1]:
import ms3
import pandas as pd
import numpy as np
import math
import fractions
import matplotlib.pyplot as plt

## 0. Dataset preparation

In [2]:
%%time

# Suppress SettingWithCopy warnings
pd.options.mode.chained_assignment = None

chorales_dataset = pd.read_csv('DM 2022 - Assignment 3 [Dataset].csv', converters = {'duration': fractions.Fraction, 'nominal_duration': fractions.Fraction, 'scalar': fractions.Fraction}, index_col = 0, low_memory = False)

# Ignore gracenotes (there is only one)
chorales_dataset = chorales_dataset[chorales_dataset['gracenote'].isna()].drop(columns = ['gracenote'])

# Remove Chorale043 because it is corrupted (it does not open in MuseScore and it has only 1 staff and no closing fermata)
chorales_dataset.drop(chorales_dataset[chorales_dataset['piece']=='BachChorales/Chorale043'].index, inplace=True)

# Get unique piece/staff pairs
piece_staffs = chorales_dataset[['piece', 'staff']].drop_duplicates().to_numpy()

# Get only first or first 
fermata_data = []
for piece, staff in piece_staffs:
    for idx, row in chorales_dataset[(chorales_dataset['piece'] == piece) & (chorales_dataset['staff'] == staff)].iterrows():
        fermata_data.append(row)
        if row['fermata'] and row['mn'] >= 4:
            break

fermata_df = pd.DataFrame(fermata_data)

CPU times: user 11.5 s, sys: 208 ms, total: 11.7 s
Wall time: 12 s


In [5]:
fermata_df

,piece,mn,mn_onset,timesig,act_dur,staff,voice,duration,nominal_duration,scalar,tied,tpc,midi,fermata
19,BachChorales/Chorale237,1,3/4,4/4,1,3,1,3/16,1/8,3/2,-1.0,2,62,False
23,BachChorales/Chorale237,1,15/16,4/4,1,3,1,1/16,1/16,1,NaN,0,60,False
25,BachChorales/Chorale237,2,0,4/4,1,3,1,1/8,1/8,1,NaN,-2,58,False
28,BachChorales/Chorale237,2,1/8,4/4,1,3,1,1/8,1/8,1,NaN,2,62,False
32,BachChorales/Chorale237,2,1/4,4/4,1,3,1,1/8,1/8,1,NaN,0,60,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,BachChorales/Chorale189,3,3/4,4/4,1,2,1,1/4,1/4,1,NaN,8,68,False
66,BachChorales/Chorale189,4,0,4/4,1,2,1,1/4,1/4,1,NaN,8,68,False
71,BachChorales/Chorale189,4,1/4,4/4,1,2,1,1/8,1/8,1,NaN,6,66,False
73,BachChorales/Chorale189,4,3/8,4/4,1,2,1,1/8,1/8,1,NaN,3,69,False
